In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import numpy as np
from lib.graph_local_classes import GraphStructure, GraphParams, InnerGraphSimulation

In [3]:

from lib.graph_enumerator import generate_graphs
from lib.node_semantics import Node_Name_Rule, Edge_Semantics_Rule
from lib import config, result_config
from lib.likelihood_calculations_shared_params import Inference
from lib.utils import filename_utility
import time

In [4]:
graph_iter = generate_graphs(**config.generator_dictionary)
graphs = list(graph_iter)
for graph in graphs:
    Node_Name_Rule.graph_semantics_apply(graph,config.node_semantics)
    Edge_Semantics_Rule.graph_semantics_apply(graph,config.edge_semantics)

inference_obj = Inference()



In [5]:
result_graphs, result_posterior, result_loglik, result_dict = inference_obj.p_graph_given_d(graphs,config.options)


JoblibTypeError: JoblibTypeError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/local/Cellar/python3/3.5.0/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    165         sys.exit(msg)
    166     main_globals = sys.modules["__main__"].__dict__
    167     if alter_argv:
    168         sys.argv[0] = mod_spec.origin
    169     return _run_code(code, main_globals, None,
--> 170                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...b/python3.5/site-packages/ipykernel/__main__.py')
    171 
    172 def run_module(mod_name, init_globals=None,
    173                run_name=None, alter_sys=False):
    174     """Execute a module's code without importing it

.........................................................

In [10]:
test = []

In [13]:
small_val = .05

cond1 = [0,small_val,small_val,small_val]
cond2 = [0,1,3,2]
cond3 = [0,3,2,1]
cond4 = [0,1,2,2]

conds = np.array([cond1,cond2,cond3,cond4])

array([ 0.05,  1.  ,  3.  ,  1.  ])

In [14]:
max_graph = graphs[0]
num_graphs = len(graphs)
num_params = config.options["param_sample_size"]
loglikelihood_by_param = np.zeros(shape = (num_params,num_graphs))
max_graph_params = GraphParams.from_networkx(max_graph) # fix this when you can
max_graph_params.sample()
blah = inference_obj.subgraph_loglik(graphs[1],max_graph_params,options=config.options)


> /Users/cocosci/Dropbox/Work/Berkeley/Projects/Time/hidden_structure_inference/lib/graph_local_classes.py(127)subgraph_copy()
    126         import ipdb; ipdb.set_trace()
--> 127         assert sorted([tuple(x) for x in np.array(self.names)[match_idx]]) == sorted(edge_list)
    128         tmp_gp.n = len(edge_list)

ipdb> self
ipdb> self.psi
array([ 0.0346601 ,  0.29583947,  0.07826709,  0.11707177,  0.43367587,
        1.47998856,  0.40990862,  0.15834476,  0.042935  ,  0.76834801,
        0.03106284,  0.10213278,  0.00920209,  0.1237447 ,  0.51181212])
ipdb> q


BdbQuit: 

In [8]:
result_graphs